<a href="https://colab.research.google.com/github/Noam-Coh3n/ModCrypto/blob/main/ProgrammingAssignmentWeek4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Oracle

In [20]:
import numpy as np
mac_sock = None
vrfy_sock = None

def Oracle_Connect():
    import socket
    global mac_sock
    global vrfy_sock
    mac_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    vrfy_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        mac_sock.connect(('128.8.130.16', 49102))
        vrfy_sock.connect(('128.8.130.16', 49103))
    except socket.error as e:
        print(e)
        return -1

    print("Connected to server successfully.")

    return 0

def Oracle_Disconnect():
    if not mac_sock or not vrfy_sock:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1

    mac_sock.close()
    vrfy_sock.close()

    print("Connection closed successfully.")

    return 0

# Packet Structure: < mlength(1) || message(mlength) || null-terminator(1) >
def Mac(message, mlength):
    if not mac_sock or not vrfy_sock:
        print("[WARNING]: You haven't connected to the server yet.")
        return -1

    out = bytearray(message[:])
    out.insert(0, mlength)
    out.append(0)

    mac_sock.send(bytearray(out))

    tag = mac_sock.recv(16)

    return bytearray(tag)

# Packet Structure: < mlength(1) || message(mlength) || tag(16) || null-terminator(1) >
def Vrfy(message, mlength, tag):
    if not mac_sock or not vrfy_sock:
        print("[WARNING]: You haven't conected to the server yet.")
        return -1

    out = bytearray(message[:])
    out.insert(0, mlength)
    out += tag
    out.append(0)

    vrfy_sock.send(bytearray(out))

    match = vrfy_sock.recv(2)
    
    return int(match.decode('ascii').strip('\0'))


#Solution
For an explanation of the solution, see the homework pdf.

In [35]:
def xor_block(b1, b2):
  """Returns the element-wise XOR of two lists."""
  return [c1 ^ c2 for c1, c2 in zip(b1,b2)]

def forge(m_forge, L):
  """
  Forge a tag for 2r-block message m_forge using queries on r-block messages.
  """
  r = len(m_forge) // L // 2
  m = [m_forge[i*L : (i+1)*L] for i in range(2*r)]  # Split message
  t = Mac(sum(m[:r], []), r*L)

  return Mac(xor_block(t, m[r]) + sum(m[r+1:], []), r*L)


In [43]:
# Your goal is to forge a valid tag on the 4-block (64-byte) message:
m_forge = "I, the server, hereby agree that I will pay $100 to this student"
m_forge_enc = [ord(c) for c in m_forge]

L = len(m_forge) // 4

Oracle_Connect()

t = forge(m_forge_enc, L)

ret = Vrfy(m_forge_enc, 4*L, t)

if (ret == 1):
    print(f"Message {m_forge!r} verified successfully!")
else:
    print("Message verification failed.")

Oracle_Disconnect()

Connected to server successfully.
Message 'I, the server, hereby agree that I will pay $100 to this student' verified successfully!
Connection closed successfully.


0